In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import xgboost as xgb

from sklearn.model_selection import train_test_split


In [ ]:
# Dataset Preparation
print ("Read Dataset ... ")
train =pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv",parse_dates=['date'],  index_col="date")

test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",parse_dates=['date'],  index_col="date")

# Text Data Features
print ("Prepare text data of Train and Test ... ")

In [ ]:
train.head()

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
sb.set(font_scale=1.2)
sb.distplot(train['num_sold'],color='violet')
plt.xlabel('num_sold',fontsize=20)
print('Skew Dist:',train['num_sold'].skew())
print('Kurtosis Dist:',train['num_sold'].kurt())

In [ ]:
train[['num_sold']].describe()

In [ ]:
train[train.num_sold>2000].shape[0]

In [ ]:
train['num_sold']=train.apply(lambda row: 2000 if row["num_sold"] >2000 else row["num_sold"] , axis=1)

In [ ]:
train.head()

In [ ]:
train['day_of_month'] = train.index.day
train['day_of_week'] = train.index.dayofweek
train['month'] = train.index.month
train['weekend'] = train.apply(lambda row: 0 if row["day_of_week"] <5 else 1 , axis=1)
train['quarter'] = train.index.quarter
train['dayofyear'] = train.index.dayofyear


In [ ]:
train['year'] = train.index.year

In [ ]:

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
train['country'] = LE.fit_transform(train['country'])
train['store'] = LE.fit_transform(train['store'])
train['product'] = LE.fit_transform(train['product'])


In [ ]:
x=train[['country','store','product','month','day_of_month','day_of_week','weekend','quarter','dayofyear']]
y=train['num_sold'].astype('float32')

In [ ]:
import seaborn as sns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()
plt.figure(figsize=(20,9))

ax = sns.pointplot(data=train,x='month',y='num_sold',hue="year")

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=250,
                          depth=16,
                          learning_rate=0.1,
                           task_type="GPU",
                           )
model.fit(x, y)

In [ ]:
test['day_of_month'] = test.index.day
test['day_of_week'] = test.index.dayofweek
test['month'] = test.index.month
test['weekend'] = test.apply(lambda row: 0 if row["day_of_week"] <5 else 1 , axis=1)
test['quarter'] = test.index.quarter
test['dayofyear'] = test.index.dayofyear

In [ ]:
test['year'] = test.index.year

In [ ]:
test['country'] = LE.fit_transform(test['country'])
test['store'] = LE.fit_transform(test['store'])
test['product'] = LE.fit_transform(test['product'])
X_test=test[['country','store','product','month','day_of_month','day_of_week','weekend','quarter','dayofyear']]


In [ ]:
ypred = model.predict(X_test)
test['num_sold']=np.round(ypred,decimals=0).astype('int')


In [ ]:
print ("Generate Submission File ... ")
test[['row_id','num_sold']].to_csv("submission.csv",index=False)